# 2.1 Calculation of deterministic results (2030)

## Importing libraries

In [ ]:
from brightway2 import *
import os     
import numpy as np     
import pandas as pd     
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import string
import pickle
%matplotlib inline

## Projects setting

In [ ]:
projects.set_current('pLCA_RegAC_2030')

## ImpactWorld+ import

In [ ]:
# Import the specified BW2Package for the LCIA method IMPACTWorld+ file from the given path or import it to the same file location as this notebook
BW2Package.import_file("Brightway_IW_damage_1_46_and_midpoint_1_28.bw2package")

# The water use method needs to be updated due to the fact that it was wrongly created.
# Filter the methods list to find the desired method for 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)'.
# Then, load the method and its characterization factors into the WS_lst variable.
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]
WS = Method(IWP_mid[9])
WS_lst = WS.load()

# Define and remove certain characterization factors (CF) from the WS_lst.
ex_CF = (('biosphere3', '2404b41a-2eed-4e9d-8ab6-783946fdf5d6'), -42.95353086694035)
ex_CF_ocean = (('biosphere3', '4f0f15b3-b227-4cdc-b0b3-6412d55695d5'), 0)
WS_lst.remove(ex_CF)
WS_lst.remove(ex_CF_ocean)

# Define a new characterization factor and add it to the WS_lst.
new_CF = (('biosphere3', '8c1494a5-4987-4715-aa2d-1908c495f4eb'), 42.95353086694035)
WS_lst.append(new_CF)

# Create a new Method instance for the updated 'Water scarcity' method.
# Register the new method and write the updated characterization factors to it.
new_WS = Method(('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Water scarcity'))
new_WS.register()
new_WS.write(WS_lst)

Assigning methods and units required for axis labels in visualisations

In [ ]:
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]
units_IWP_mid=['kg CO2eq.','kg CO2eq.','MJ dep.','kg dep.', 'kg NMVOCeq.','kg CFC-11eq.','CTUe','CTUh','CTUh','m3eq.','kg SO2eq.','kg SO2eq.','kg PO4eq.','kg Neq.', 'm2eq.', 'm2eq*yr.', 'kg PM2.5eq.', 'Bq C14eq.']

In [ ]:
IWP_end = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Damage 1.46)' in m[0]]

## Selection of processes for the configurations

Defining the 'use of aircraft' processes for all configurations, SSP scenarios and fuels from the imported LCI databases

In [ ]:
#Aircraft use processes for the typical mission (200 nmi)

#KERSOSENE FUELED AIRCRAFT
#conventional 
typ_aircraft_conv_kero_NDC = Database('GENESIS_2030_conventional_NDC').get('F2FEAAE907F648A69BFB501E91366D35')
typ_aircraft_conv_kero_Base = Database("GENESIS_2030_conventional_Base").get('F2FEAAE907F648A69BFB501E91366D35')
typ_aircraft_conv_kero_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('F2FEAAE907F648A69BFB501E91366D35')

#Gt-bat 
typ_aircraft_GT_bat_kero_NDC = Database("GENESIS_2030_GT-bat_NDC").get('6F97863F6BEF4DE383907C08DE912E6D')
typ_aircraft_GT_bat_kero_Base = Database("GENESIS_2030_GT-bat_Base").get('6F97863F6BEF4DE383907C08DE912E6D')
typ_aircraft_GT_bat_kero_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('6F97863F6BEF4DE383907C08DE912E6D')


#AAF FUELED AIRCRAFT
#Conventional
typ_aircraft_conv_AAF_NDC = Database("GENESIS_2030_conventional_NDC").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_AAF_Base = Database("GENESIS_2030_conventional_Base").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_AAF_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_ILUC_NDC = Database("GENESIS_2030_conventional_NDC_ILUC").get('BE40C283116D46E986C246F2B77A86D7')

#Gt-bat
typ_aircraft_GT_bat_AAF_NDC = Database("GENESIS_2030_GT-bat_NDC").get('DFDABF367DB344E4AFEF072D2A7634F3')
typ_aircraft_GT_bat_AAF_Base = Database("GENESIS_2030_GT-bat_Base").get('DFDABF367DB344E4AFEF072D2A7634F3')
typ_aircraft_GT_bat_AAF_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('DFDABF367DB344E4AFEF072D2A7634F3')
typ_aircraft_GT_bat_ILUC_NDC = Database("GENESIS_2030_GT-bat_NDC_ILUC").get('DFDABF367DB344E4AFEF072D2A7634F3')


In [ ]:
#Aircraft use processes for the design mission (600 nmi)

#KERSOSENE FUELED AIRCRAFT
#conventional
des_aircraft_conv_kero_NDC = Database('GENESIS_2030_conventional_NDC').get('0808F060E0F94D1BB0380DFA4A9065F5')
des_aircraft_conv_kero_Base = Database("GENESIS_2030_conventional_Base").get('0808F060E0F94D1BB0380DFA4A9065F5')
des_aircraft_conv_kero_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('0808F060E0F94D1BB0380DFA4A9065F5')

#Gt-bat
des_aircraft_GT_bat_kero_NDC = Database("GENESIS_2030_GT-bat_NDC").get('3FCE85DD88EF4504A6F7D6326303F33D')
des_aircraft_GT_bat_kero_Base = Database("GENESIS_2030_GT-bat_Base").get('3FCE85DD88EF4504A6F7D6326303F33D')
des_aircraft_GT_bat_kero_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('3FCE85DD88EF4504A6F7D6326303F33D')

#AAF FUELED AIRCRAFT
#Conventional
des_aircraft_conv_AAF_NDC = Database("GENESIS_2030_conventional_NDC").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_AAF_Base = Database("GENESIS_2030_conventional_Base").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_AAF_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_ILUC_NDC = Database("GENESIS_2030_conventional_NDC_ILUC").get('BA3B99FBFBDA4677867DD3202F1BD7CC')

#Gt-bat
des_aircraft_GT_bat_AAF_NDC = Database("GENESIS_2030_GT-bat_NDC").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_AAF_Base = Database("GENESIS_2030_GT-bat_Base").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_AAF_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_ILUC_NDC = Database("GENESIS_2030_GT-bat_NDC_ILUC").get('8C441EC088644DCAA1FF915F3E9AF97F')



Setting up dictionary with all 2030 typical mission configurations and SSP scenarios

In [ ]:
configurations_typ = {
    'typ_aircraft_conv_kero_Base': typ_aircraft_conv_kero_Base,
    'typ_aircraft_conv_kero_NDC': typ_aircraft_conv_kero_NDC,
    'typ_aircraft_conv_kero_PkBudg500': typ_aircraft_conv_kero_PkBudg500,
    'typ_aircraft_GT_bat_kero_Base': typ_aircraft_GT_bat_kero_Base,
    'typ_aircraft_GT_bat_kero_NDC': typ_aircraft_GT_bat_kero_NDC,
    'typ_aircraft_GT_bat_kero_PkBudg500': typ_aircraft_GT_bat_kero_PkBudg500,
    'typ_aircraft_conv_AAF_Base': typ_aircraft_conv_AAF_Base,
    'typ_aircraft_conv_AAF_NDC': typ_aircraft_conv_AAF_NDC,
    'typ_aircraft_conv_AAF_PkBudg500': typ_aircraft_conv_AAF_PkBudg500,
    'typ_aircraft_GT_bat_AAF_Base': typ_aircraft_GT_bat_AAF_Base,
    'typ_aircraft_GT_bat_AAF_NDC': typ_aircraft_GT_bat_AAF_NDC,
    'typ_aircraft_GT_bat_AAF_PkBudg500': typ_aircraft_GT_bat_AAF_PkBudg500,
    'typ_aircraft_GT_bat_ILUC_NDC':typ_aircraft_GT_bat_ILUC_NDC,
    'typ_aircraft_conv_ILUC_NDC':typ_aircraft_conv_ILUC_NDC}

Setting up dictionary with all 2030 design mission configurations and SSP scenarios

In [ ]:
configurations_des= {
    'des_aircraft_conv_kero_Base': des_aircraft_conv_kero_Base,
    'des_aircraft_conv_kero_NDC': des_aircraft_conv_kero_NDC,
    'des_aircraft_conv_kero_PkBudg500': des_aircraft_conv_kero_PkBudg500,
    'des_aircraft_GT_bat_kero_Base': des_aircraft_GT_bat_kero_Base,
    'des_aircraft_GT_bat_kero_NDC': des_aircraft_GT_bat_kero_NDC,
    'des_aircraft_GT_bat_kero_PkBudg500': des_aircraft_GT_bat_kero_PkBudg500,
    'des_aircraft_conv_AAF_Base': des_aircraft_conv_AAF_Base,
    'des_aircraft_conv_AAF_NDC': des_aircraft_conv_AAF_NDC,
    'des_aircraft_conv_AAF_PkBudg500': des_aircraft_conv_AAF_PkBudg500,
    'des_aircraft_GT_bat_AAF_Base': des_aircraft_GT_bat_AAF_Base,
    'des_aircraft_GT_bat_AAF_NDC': des_aircraft_GT_bat_AAF_NDC,
    'des_aircraft_GT_bat_AAF_PkBudg500': des_aircraft_GT_bat_AAF_PkBudg500,
    'des_aircraft_GT_bat_ILUC_NDC':des_aircraft_GT_bat_ILUC_NDC,
    'des_aircraft_conv_ILUC_NDC':des_aircraft_conv_ILUC_NDC}

## Deterministic results

### Midpoint

Setting up ImpactWorld+ as midpoint LCIA method 

In [ ]:
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]

#### Typical mission

Setting up empty dataframe for midpoint results

In [ ]:
deterministic_mid_typ = pd.DataFrame(columns=[col[1] for col in IWP_mid], index=list(configurations_typ.keys()))

Generating midpoint results for all configurations with the typical mission with ImpactWorld+ and saving them in a pickle format

In [ ]:
# generating midpoint results for 18 categories, for all 14 configurations

for j in range(18):
    for i in range(14):
        test=configurations_typ[deterministic_mid_typ.index[i]]
        E=[test]
        fu={E[0]:1}
        lca = LCA(fu, IWP_mid[j])
        lca.lci()
        lca.lcia()

        #storing the midpoints in a dataframe
        deterministic_mid_typ.loc[deterministic_mid_typ.index[i],deterministic_mid_typ.columns[j]]=lca.score
display(deterministic_mid_typ)

#saving the dataframe as a pickle file
with open(f"deterministic_midpoint_typ_2030.pkl", "wb+") as f:
        pickle.dump(deterministic_mid_typ, f)

#### Design mission

Setting up empty dataframe for midpoint results

In [ ]:
deterministic_mid_des = pd.DataFrame(columns=[col[1] for col in IWP_mid], index=list(configurations_des.keys()))

Generating midpoint results for all configurations with the design mission with ImpactWorld+ and saving them in a pickle format

In [ ]:
# generating midpoint results for 18 categories, for all 14 configurations

for j in range(18):
    for i in range(14):
        test=configurations_des[deterministic_mid_des.index[i]]
        E=[test]
        fu={E[0]:1}
        lca = LCA(fu, IWP_mid[j])
        lca.lci()
        lca.lcia()

        #storing the midpoints in a dataframe
        deterministic_mid_des.loc[deterministic_mid_des.index[i],deterministic_mid_des.columns[j]]=lca.score
display(deterministic_mid_des)

#saving the dataframe as a pickle file
with open(f"deterministic_midpoint_des_2030.pkl", "wb+") as f:
        pickle.dump(deterministic_mid_des, f)

### Endpoint

Setting up ImpactWorld+ as endpoint LCIA method 

In [ ]:
IWP_end = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Damage 1.46)' in m[0]]

#### Typical mission

Setting up empty dataframe for endpoint results

In [ ]:
deterministic_end_typ = pd.DataFrame(columns=[col[1] for col in IWP_end], index=list(configurations_typ.keys()))

Generating endpoint results for all configurations with ImpactWorld+ and saving them in a pickle format

In [ ]:
# generating endpoint results for 27 categories, for all 14 configurations

for j in range(27):
    for i in range(14):
        test=configurations_typ[deterministic_end_typ.index[i]]
        E=[test]
        fu={E[0]:1}
        lca = LCA(fu, IWP_end[j])
        lca.lci()
        lca.lcia()

         #storing the endpoints in a dataframe
        deterministic_end_typ.loc[deterministic_end_typ.index[i],deterministic_end_typ.columns[j]]=lca.score
display(deterministic_end_typ)

#saving the dataframe as a pickle file
with open(f"deterministic_endpoint_typ_2030.pkl", "wb+") as f:
        pickle.dump(deterministic_end_typ, f)

#### Design mission

Setting up empty dataframe for endpoint results

In [ ]:
deterministic_end_des = pd.DataFrame(columns=[col[1] for col in IWP_end], index=list(configurations_des.keys()))

Generating endpoint results for all configurations with ImpactWorld+ and saving them in a pickle format

In [ ]:
# generating endpoint results for 27 categories, for all 14 configurations

for j in range(27):
    for i in range(14):
        test=configurations_des[deterministic_end_des.index[i]]
        E=[test]
        fu={E[0]:1}
        lca = LCA(fu, IWP_end[j])
        lca.lci()
        lca.lcia()

         #storing the endpoints in a dataframe
        deterministic_end_des.loc[deterministic_end_des.index[i],deterministic_end_des.columns[j]]=lca.score
display(deterministic_end_des)

#saving the dataframe as a pickle file
with open(f"deterministic_endpoint_des_2030.pkl", "wb+") as f:
        pickle.dump(deterministic_end_des, f)

## Contribution of life cycle stages 

The contribution of life cycle stages is only performed within the NDC scenario for endpoint.

Retrieving all 'aircraft usage' proces codes from the NDC databases

In [ ]:
acts_conv_2030 = [act['code'] for act in Database('GENESIS_2030_conventional_NDC') if act['name'].startswith('aircraft usage')]
acts_GT_bat_2030 = [act['code'] for act in Database('GENESIS_2030_GT-bat_NDC') if act['name'].startswith('aircraft usage')]

Set up function for contributiona analysis calculation

In [ ]:
def ca_calc(act_list, database, method):
    ca_dict = {}
    for act in act_list:
        test = database.get(act)
        E = [test]
        fu = {E[0]: 1}
        lca = LCA(fu, method)
        lca.lci()
        lca.lcia()
        ca_list = []
        ca_list.append(('Combustion', lca.characterized_inventory[:,lca.activity_dict.get(test.key)].sum()))
        for exc in test.technosphere():
            lca.redo_lcia({exc.input: exc['amount']})
            ca_list.append((exc.input['name'], lca.score))
            ca_dict[test['name']] = ca_list
    return ca_dict

Selection of relevant endpoint methods. For the climate change, human toxicity, marine acidification and freshwater ecotoxicicty damages, the impacts are only considered over the short-term time span (from t=0 to t=100 years). See  world impact+ article from Bulle et al.(2019)

In [ ]:
HH= [IWP_end[2], IWP_end[7], IWP_end[11],IWP_end[12],IWP_end[13],IWP_end[16],IWP_end[18],IWP_end[25]]
EQ=[IWP_end[0], IWP_end[4], IWP_end[6],IWP_end[8],IWP_end[9],IWP_end[10],IWP_end[14],IWP_end[20],IWP_end[21],IWP_end[22],IWP_end[23],IWP_end[24],IWP_end[26]]

### Human Health

In [ ]:
#creating dataframe
summed_df =pd.DataFrame()

#running the life cycle stages contribution for all categories that contribute to the human health area of concern. 
for IC in HH:
    ca = ca_calc(acts_GT_bat_2030, Database('GENESIS_2030_GT-bat_NDC'), IC) | ca_calc(acts_conv_2030, Database('GENESIS_2030_conventional_NDC'), IC)
    ca_data = [[k, *v] for k, lst in ca.items() for v in lst]

    #setting up dataframe
    df = pd.DataFrame(ca_data, columns=['Configuration', 'Stage',  str(IC)])
    df = df.groupby(['Configuration','Stage'])[ str(IC)].sum().unstack().fillna(0)
    df['production of aircraft, GT-bat'] = df['production of aircraft, GT-bat'] + df['production of aircraft, conventional']
    df = df.drop(columns='production of aircraft, conventional')
    df = df.rename(columns={'SAF production, biodiesel proxy': 'SAF production',
                            'airport use': 'Airport use',
                            'market for kerosene': 'Kerosene production',
                            'market group for electricity, low voltage': 'Electricity production',
                            'production of aircraft, GT-bat': 'Aircraft production'})
    df.index = pd.MultiIndex.from_tuples(df.index.str.split(', ').tolist())
    df = df.reset_index(names=['LC', 'Mission', 'Configuration', 'Fuel']).drop(columns=['LC']).fillna(value='Kerosene')
    df = df.rename_axis(None, axis=1)
    df['Configuration'] = df['Configuration'].replace('conventional', 'Conventional')
    if not summed_df.empty:
          summed_df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']] = summed_df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']] + df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']]
    else:
          summed_df = df
            
display(summed_df)       

#saving dataframe as pickle file
with open(f"endpoint_stages_contribution_HH_2030.pkl", "wb+") as f:
        pickle.dump(summed_df, f)

### Ecosystem Quality

In [ ]:
#creating dataframe
summed_df =pd.DataFrame()

#running the life cycle stages contribution for all categories that contribute to the ecosystem quality area of concern. 
for IC in EQ:
    ca = ca_calc(acts_GT_bat_2030, Database('GENESIS_2030_GT-bat_NDC'), IC) | ca_calc(acts_conv_2030, Database('GENESIS_2030_conventional_NDC'), IC)
    ca_data = [[k, *v] for k, lst in ca.items() for v in lst]

    #setting up dataframe
    df = pd.DataFrame(ca_data, columns=['Configuration', 'Stage', str(IC)])
    df = df.groupby(['Configuration','Stage'])[str(IC)].sum().unstack().fillna(0)
    df['production of aircraft, GT-bat'] = df['production of aircraft, GT-bat'] + df['production of aircraft, conventional']
    df = df.drop(columns='production of aircraft, conventional')
    df = df.rename(columns={'SAF production, biodiesel proxy': 'SAF production',
                            'airport use': 'Airport use',
                            'market for kerosene': 'Kerosene production',
                            'market group for electricity, low voltage': 'Electricity production',
                            'production of aircraft, GT-bat': 'Aircraft production'})
    df.index = pd.MultiIndex.from_tuples(df.index.str.split(', ').tolist())
    df = df.reset_index(names=['LC', 'Mission', 'Configuration', 'Fuel']).drop(columns=['LC']).fillna(value='Kerosene')
    df = df.rename_axis(None, axis=1)
    df['Configuration'] = df['Configuration'].replace('conventional', 'Conventional')
    if not summed_df.empty:
          summed_df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']] = summed_df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']] + df[['Combustion','SAF production', 'Airport use', 'Kerosene production', 'Electricity production', 'Aircraft production']]
    else:
          summed_df = df
            
display(summed_df)

#saving dataframe as pickle file
with open(f"endpoint_stages_contribution_EQ_2030.pkl", "wb+") as f:
        pickle.dump(summed_df, f)